# Data Cleaning

It is always amazing to see how insights can be generated by data scientists. However, in real life, not all collected data sets can be directly analyzed by established tools, primarily due to the chaotic nature of data. Data scientists spend 80% of their time on simply cleaning up the data. Therefore, it is important to understand how to manage messy data in order to transform them into organized data that are ready for further processing.

In this web page, we will be discussing eight basic steps on data cleaning.

In [13]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

mydata = pd.read_csv("data/Building_Permits.csv")

In [3]:
mydata.head(1)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945


Here, we have our first glimpse of the data. It is obvious that there are several "NaN" values existing in the dataset. We definitely do not want to have NA values in our dataset. Before tackling these NA values (in Step 2), we will begin our data cleaning process by getting rid of extra spaces in the data.

## Step 1 : Get Rid Of Extras Spaces

In [4]:
string = "   hello worlds     "

print(string)
print()
print("="*100)

fresh_string = string.strip()
print()
print(fresh_string)

   hello worlds     


hello worlds


The simple example illustrates that when there are extra spaces before and after the string, we want to ensure that we are getting rid of these unnecessary spaces. We can apply the same methodology to the entire data set.


---
## Step 2 : Remove All NA Values

The second step is the most obvious step that we should do when it comes to cleaning our data set: removing the NA values that exist within the dataset. To do so, we can apply one of several techniques.

The first method is fast and easy. We can simply substitute all the NA values for 0.

In [26]:
test_row = mydata.iloc[0:5]
test_row.fillna(0, inplace=True)
test_row.head(1)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,0,Ellis,St,...,3.0,constr type 3,0.0,0,0,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945


However, this methodology has its limitations. For example, we can see here that the Proposed Construction Type Description cannot be 0, so this would be inappropriate to treat this NA value as a 0. Being a description, the value here should be a string instead of number. So how do we want to handle scenarios where the data in a column are strings?

There is a simple methodology that can be applied by substituting the NA value in Proposed Construction Type Description with the value that appears most frequently in that column.

In [24]:
test_row = mydata.iloc[0:100]
test_row['Proposed Construction Type Description'].fillna(test_row['Proposed Construction Type Description'].value_counts()[:1].index.tolist()[0],inplace=True)
test_row.head(5)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,0,Ellis,St,...,3.0,constr type 3,0.0,0,0,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,0,Geary,St,...,3.0,constr type 3,0.0,0,0,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,0,Pacific,Av,...,1.0,constr type 1,1.0,constr type 1,0,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,0,Pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),0,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,0,Market,St,...,3.0,constr type 3,0.0,0,0,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992


Notice how some of NAs in the Proposed Construction Type Description have now successfully changed to "wood frame(5)", which is the most frequent value of that column.

---
## Step 3 : Convert String Numbers Into Real Numbers

Trying to convert strings into numbers in Excel can be a messy process; however, using Python, this conversion process is really simple to acheive it. Here, we introcude two simple commands: "float" and "int".

In [30]:
test_row = mydata.iloc[0:5]
test_row['Permit Type'].astype(float)

0    4.0
1    4.0
2    3.0
3    8.0
4    6.0
Name: Permit Type, dtype: float64

In [31]:
test_row = mydata.iloc[0:5]
test_row['Permit Type'].astype(int)

0    4
1    4
2    3
3    8
4    6
Name: Permit Type, dtype: int64

The "float" command converts a number so that it appears in scientific notation, while the "int" command converts a decimal number into an integer by rounding down. These two commands allow for flexibility in application.

---
## Step 4 : Remove Unnecessary Duplicates

Sometimes, rows with same content will appear twice in our dataset, which is most likely undesirable for our dataset. How can we handle these unnecessary duplicates?

In [36]:
test_row = mydata.iloc[0:2]
test_row2 = mydata.iloc[0:2]
frame = [test_row,test_row2]
test_df = pd.concat(frame);test_df.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,0,Ellis,St,...,3.0,constr type 3,0.0,0,0,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,0,Geary,St,...,3.0,constr type 3,0.0,0,0,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,0,Ellis,St,...,3.0,constr type 3,0.0,0,0,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,0,Geary,St,...,3.0,constr type 3,0.0,0,0,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718


Here, we see two rows that have been duplicated, and we would like to remove these duplicate rows. To do so, we use the "drop_duplicates" command to achieve our desired result: a data set that is free of unnecessary duplicates.

In [37]:
test_df.drop_duplicates()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,0,Ellis,St,...,3.0,constr type 3,0.0,0,0,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,0,Geary,St,...,3.0,constr type 3,0.0,0,0,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718


---
## Step 5 : Error Handling

The error seen in the following example is not the typical error seen in Python. This error refers more to a logical error in the data set, which we can better illustrate with the example below.

In [39]:
test_row = mydata.iloc[0:5]
test_row.head(1)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,0,Ellis,St,...,3.0,constr type 3,0.0,0,0,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945


In the example, we have an Existing Construction Type value of 3.0, with the Existing Construction Type Description being "constr type 3". This is a fixed relationship; any rows with different a value can be considered an "error", which we need to fix when we are cleaning the data.

In [62]:
test_row["Existing Construction Type"].unique()
print("There are 3 construction Types which are 1, 3 and 5")
print()
print("="*100)
print()

def errorcheck(a):
    if a["Existing Construction Type"] == 3.0:
        a["Existing Construction Type Description"] = "constr type 3"
    if a["Existing Construction Type"] == 1.0:
        a["Existing Construction Type Description"] = "constr type 1"
    if a["Existing Construction Type"] == 5.0:
        a["Existing Construction Type Description"] = "wood frame (5)"

There are 3 construction Types which are 1, 3 and 5




In [67]:
for index, row in test_row.iterrows():
    errorcheck(row)
test_row.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,0,ellis,St,...,3.0,constr type 3,0.0,0,0,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,0,geary,St,...,3.0,constr type 3,0.0,0,0,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,0,pacific,Av,...,1.0,constr type 1,1.0,constr type 1,0,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,0,pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),0,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,0,market,St,...,3.0,constr type 3,0.0,0,0,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992


---
## Step 6 : Upper/Lower Cases

There are many reasons that why upper or lower cases should be considered in data cleaning. There may have been human errors while inputing the data, and some of NLP algorithms consider upper cases in data as a factor in expressing strong opinions. Therefore, we want to ensure that we are being cautious with the usage of upper and/or lower cases in our data set.

One of the universal ways to remove the case influence in data is to first change all values to upper case, and then change them all back to lower case again. This ensures that all our data are, in fact, consistently in lower case.

In [44]:
test_row = mydata.iloc[0:5]
test_row["Street Name"] = test_row["Street Name"].str.upper();print(test_row["Street Name"] )

0      ELLIS
1      GEARY
2    PACIFIC
3    PACIFIC
4     MARKET
Name: Street Name, dtype: object


In [45]:
test_row["Street Name"] = test_row["Street Name"].str.lower();print(test_row["Street Name"] )

0      ellis
1      geary
2    pacific
3    pacific
4     market
Name: Street Name, dtype: object


Using this universal method, notice how there are no more upper cases left in the dataframe. The NLP algorithm can be applied perfectly to this data set.

---
## Step 7 : Stop Words/Spell Check

Stop words are words in the English language that have very little meaning, such as "the", "a", "is", etc. Filtering out stop words from strings in a data set is primarily done in special scenarios where we want to perform language processing on the data. We can remove these stop words from the data set by doing the following:

In [70]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

text = "I have a lot of games to play with, do you want to play with me?"
text_tokens = word_tokenize(text)

tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]

print(tokens_without_sw)

['I', 'lot', 'games', 'play', ',', 'play', '?']


---
## Step 8 : Formatting Removal

Removing formatting from the data set mainly deals with the nature of the data set. Sometimes, the second-hand data set that is being used was originally formatted and designed for a specific type of analysis, so the data set may contain features that are unnecessary or even undesirable for our analytical purposes, such as including a heading in the first row. In this way, we may want to remove the existing formatting in the data set. However, this cleaning process will vary on a case-by-case basis, making it difficult to illustrate an example here. Therefore, we are only mentioning that the removal of any existing formatting from the data set is a possibility in the data cleaning process.

---
## Homework!
Can you clean the building permit data set so that it is ready for regression analysis?

### References

https://www.digitalvidya.com/blog/data-cleaning-techniques/ 


https://www.kaggle.com/rtatman/data-cleaning-challenge-handling-missing-values


https://www.geeksforgeeks.org/python-pandas-dataframe-drop_duplicates/


https://stackabuse.com/removing-stop-words-from-strings-in-python/#:~:text=To%20remove%20stop%20words%20from%20a%20sentence%2C%20you%20can%20divide,stop%20words%20provided%20by%20NLTK.&text=In%20the%20script%20above%2C%20we,()%20method%20from%20the%20nltk.